In [86]:
import requests
page = requests.get("https://www.larvalabs.com/cryptopunks/sales")

In [87]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [88]:
links = soup.find_all('a')
titles = [link.get('title') for link in links]
titles

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Punk #682',
 'Punk #9673',
 'Punk #8351 For Sale: 23.98 ETH ($33,548.26 USD)',
 'Punk #2594',
 'Punk #3459',
 'Punk #4237',
 'Punk #8196',
 'Punk #6713 For Sale: 28.5 ETH ($39,871.79 USD)',
 'Punk #6028',
 'Punk #4631',
 'Punk #1392',
 'Punk #7072 For Sale: 24.5 ETH ($34,275.75 USD) Bid: 23 ETH ($32,177.23 USD)',
 'Punk #7405 For Sale: 28 ETH ($39,172.28 USD)',
 'Punk #7072 For Sale: 24.5 ETH ($34,275.75 USD) Bid: 23 ETH ($32,177.23 USD)',
 'Punk #3824 For Sale: 54 ETH ($75,546.54 USD)',
 'Punk #3160 For Sale: 49.99 ETH ($69,936.51 USD)',
 'Punk #8242',
 'Punk #5875',
 'Punk #5083',
 'Punk #8445 For Sale: 84.84 ETH ($118,692.01 USD)',
 'Punk #4880',
 'Punk #7342 For Sale: 69 ETH ($96,531.69 USD)',
 'Punk #6669 For Sale: 36 ETH ($50,364.36 USD)',
 'Punk #1967 For Sale: 60 ETH ($83,940.60 USD)',
 'Punk #4802',
 'Punk #4248',
 'Punk #825 For Sale: 38.99 ETH ($54,547.40 USD)',
 'Punk #5990',
 'Punk #6516 For Sale: 29.99 ETH ($41,956.31 USD

In [89]:
import pandas as pd
forsale = pd.DataFrame({
    "Description": titles
})
df = forsale.dropna()
df = df.reset_index(drop=True)
df.head()

,Description
0,Punk #682
1,Punk #9673
2,"Punk #8351 For Sale: 23.98 ETH ($33,548.26 USD)"
3,Punk #2594
4,Punk #3459


In [90]:
texttocolumns = df['Description'].str.split(' ',expand=True)
texttocolumns = texttocolumns[[1]]
texttocolumns.columns =['CryptoPunkId']

In [91]:
texttocolumns['CryptoPunkId'] = texttocolumns['CryptoPunkId'].str.replace('#','')
texttocolumns.head()


,CryptoPunkId
0,682
1,9673
2,8351
3,2594
4,3459


In [92]:
punklist = []
result = soup.find_all("div", {"class": "text-center w-100 punk-image-text-dense"})
for results in result:
    punklist.append(results.text)
punklist

['\n                        24Ξ$33.58K\n                        ',
 '\n                        22.50Ξ$31.48K\n                        ',
 '\n                        22.10Ξ$30.92K\n                        ',
 '\n                        19.84Ξ$27.76K\n                        ',
 '\n                        23Ξ$32.18K\n                        ',
 '\n                        55Ξ$76.95K\n                        ',
 '\n                        29.99Ξ$41.96K\n                        ',
 '\n                        22Ξ$30.78K\n                        ',
 '\n                        21Ξ$29.38K\n                        ',
 '\n                        24.89Ξ$34.82K\n                        ',
 '\n                        21.99Ξ$30.76K\n                        ',
 '\n                        21.98Ξ$30.75K\n                        ',
 '\n                        21.98Ξ$30.75K\n                        ',
 '\n                        24Ξ$33.58K\n                        ',
 '\n                        44Ξ$61.56K

In [93]:
sold = pd.DataFrame({
    "Description": punklist
})
soldpunks = sold['Description'].str.split('Ξ',expand=True)
soldpunks.head()

,0,1
0,\n 24,$33.58K\n
1,\n 22.50,$31.48K\n
2,\n 22.10,$30.92K\n
3,\n 19.84,$27.76K\n
4,\n 23,$32.18K\n


In [94]:
soldpunks.columns =['ETH','Price']
soldpunks['ETH'] = soldpunks['ETH'].str.replace('\n ', '')
soldpunks['Price'] = soldpunks['Price'].str.replace('\n', '')
soldpunks.head()

,ETH,Price
0,24,$33.58K
1,22.50,$31.48K
2,22.10,$30.92K
3,19.84,$27.76K
4,23,$32.18K


In [95]:
mergeddf = texttocolumns.join(soldpunks)
mergeddf.head()

,CryptoPunkId,ETH,Price
0,682,24,$33.58K
1,9673,22.50,$31.48K
2,8351,22.10,$30.92K
3,2594,19.84,$27.76K
4,3459,23,$32.18K


In [96]:
mergeddf1 = mergeddf.groupby('CryptoPunkId')['ETH'].max()
mergeddf1 = pd.DataFrame(mergeddf1, columns = ['ETH'])
mergeddf1['CryptoPunkId'] = mergeddf1.index
mergeddf1.index = range(mergeddf1.shape[0])
maxpurchase = pd.merge(mergeddf,mergeddf1, on = ['CryptoPunkId','ETH'])

In [97]:
# maxpurchase[maxpurchase.Price.str.contains("K"|"M")]
import numpy as np
filtereddf = maxpurchase[(maxpurchase['Price'].str.contains('K')) | (maxpurchase['Price'].str.contains('M'))]
filtereddf['multiplier'] = np.where(filtereddf['Price'].str.contains('K'), 1000, 1000000)


/Users/user/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [98]:
filtereddf['Price'] = filtereddf['Price'].str.replace('$','')
filtereddf['Price'] = filtereddf['Price'].str.replace('K','')
filtereddf['Price'] = filtereddf['Price'].str.replace('M','')


/Users/user/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/user/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/user/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [99]:
filtereddf["Price"] = pd.to_numeric(filtereddf["Price"], downcast="float")
filtereddf['Amount'] = filtereddf['Price']*filtereddf['multiplier']
filtereddf

/Users/user/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/user/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,CryptoPunkId,ETH,Price,multiplier,Amount
0,9673,22.50,31.480000,1000,31479.999542
1,8351,22.10,30.920000,1000,30920.000076
2,2594,19.84,27.760000,1000,27760.000229
3,3459,23,32.180000,1000,32180.000305
4,4237,55,76.949997,1000,76949.996948
5,6713,22,30.780001,1000,30780.000687
6,6028,21,29.379999,1000,29379.999161
7,7405,21.98,30.750000,1000,30750.000000
8,3824,44,61.560001,1000,61560.001373
9,3160,40,55.959999,1000,55959.999084


In [100]:
finaldf = filtereddf[['CryptoPunkId','ETH','Amount']]

In [ ]:
finaldf.rename(columns={"Amount": "Price"})

In [101]:
finaldf.to_csv("punkssold.csv",index=False)